## Ablation study on KS dataset for PFNN consist

In [ ]:
cd ..

In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from model.utilities import *
from model.koopman_base import *
import sys
sys.path.append('./model')
from sklearn.decomposition import PCA

import numpy.random as random
from scipy.stats import gaussian_kde


font = {'size'   : 12, 'family': 'Times New Roman'}
matplotlib.rc('font', **font)

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

# Main
n_train = 1000
n_test = 100

sub = 4 # spatial subsample
S = 512
s = S//sub

T_in = 500 # skip first 100 seconds of each trajectory to let trajectory reach attractor
T = 1100 # seconds to extract from each trajectory in data
T_out = T_in + T
step = 1 # Seconds to learn solution operator

# Load data
predloader = MatReader('./data/KS.mat')
data_raw = predloader.read_field('u')
data_tensor = torch.tensor(data_raw, dtype=torch.float)[...,::sub]

# randomly sample half episodes from the train data episodes to avoid biased test distribution
episode_samples = int(0.5*n_train)
data_sampled_train = data_tensor[torch.randperm(data_tensor[:n_train].size(0))[:episode_samples],:,:]
data_test = data_tensor[-n_test:,:,:]

train_sample = data_sampled_train[:,T_in:T_out,:].reshape(-1, s)
test_a = data_test[:,T_in-1:T_out-1,:].reshape(-1, s)
test_u = data_test[:,T_in:T_out,:].reshape(-1, s)
batch_size = 100
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=batch_size, shuffle=False)

/tmp/ipykernel_6994/362903868.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(data_raw, dtype=torch.float)[...,::sub]


In [ ]:
device = torch.device('cpu')

PFNN_consist_0_path = 'fill_PFNN_gamma2_0_model_path'
model_consist_0 = torch.load(PFNN_consist_0_path, map_location=device)

PFNN_consist_01_path = 'fill_PFNN_gamma2_01_model_path'
model_consist_01 = torch.load(PFNN_consist_01_path, map_location=device)

PFNN_consist_03_path = 'fill_PFNN_gamma2_03_model_path'
model_consist_03 = torch.load(PFNN_consist_03_path, map_location=device)

PFNN_consist_05_path = 'fill_PFNN_gamma2_05_model_path'
model_consist_05 = torch.load(PFNN_consist_05_path, map_location=device)

PFNN_consist_1_path = 'fill_PFNN_gamma2_1_model_path'
model_consist_1 = torch.load(PFNN_consist_1_path, map_location=device)


In [ ]:
reduced_model1_long_pred = []
reduced_model2_long_pred = []
reduced_model3_long_pred = []
reduced_model4_long_pred = []
reduced_model5_long_pred = []

with torch.no_grad():

      for init_id in range(n_test):
            consist_0_long_pred = long_prediction(model_consist_0, test_a, init_id*T, 1, s, s, T=T)
            consist_01_long_pred = long_prediction(model_consist_01, test_a, init_id*T, 1, s, s, T=T)
            consist_03_long_pred = long_prediction(model_consist_03, test_a, init_id*T, 1, s, s, T=T)
            consist_05_long_pred = long_prediction(model_consist_05, test_a, init_id*T, 1, s, s, T=T)
            consist_1_long_pred = long_prediction(model_consist_1, test_a, init_id*T, 1, s, s, T=T)


            reduced_model1_long_pred.append(pca.transform(consist_0_long_pred))
            reduced_model2_long_pred.append(pca.transform(consist_01_long_pred))
            reduced_model3_long_pred.append(pca.transform(consist_03_long_pred))
            reduced_model4_long_pred.append(pca.transform(consist_05_long_pred))
            reduced_model5_long_pred.append(pca.transform(consist_1_long_pred))


reduced_model1_long_pred = np.concatenate(reduced_model1_long_pred, axis=0)
reduced_model2_long_pred = np.concatenate(reduced_model2_long_pred, axis=0)
reduced_model3_long_pred = np.concatenate(reduced_model3_long_pred, axis=0)
reduced_model4_long_pred = np.concatenate(reduced_model4_long_pred, axis=0)
reduced_model5_long_pred = np.concatenate(reduced_model5_long_pred, axis=0)


In [ ]:
# Step 1: Instantiate PCA object
# To keep all components but see how many are needed for 95% variance, you can set n_components to 0.95
pca = PCA(n_components=5)

# Step 2: Fit PCA on the data
pca.fit(np.concatenate((train_sample, test_u), axis=0))

# Step 3: Transform the data to get the principal components
test_u_pca = pca.transform(test_u)

print("Original shape:", test_u.shape)
print("Reduced shape:", test_u_pca.shape)
print("Number of components selected:", pca.n_components_)

Original shape: torch.Size([110000, 128])
Reduced shape: (110000, 5)
Number of components selected: 5


### Calculate KL divergence of principle components' distribution 

In [ ]:
kde_truth_pca = calculate_kde(pca.transform(np.concatenate([train_sample, test_u])))
kde_consist_0_pca = calculate_kde(reduced_model1_long_pred)
kde_consist_01_pca = calculate_kde(reduced_model2_long_pred)
kde_consist_03_pca = calculate_kde(reduced_model3_long_pred)
kde_consist_05_pca = calculate_kde(reduced_model4_long_pred)
kde_consist_1_pca = calculate_kde(reduced_model5_long_pred)

In [ ]:
kl_dim_tru_consist_0 = []
kl_dim_tru_consist_01 = []
kl_dim_tru_consist_03 = []
kl_dim_tru_consist_05 = []
kl_dim_tru_consist_1 = []
x_range = np.linspace(np.concatenate([train_sample, test_u]).min()-5, np.concatenate([train_sample, test_u]).max()+5, 40000)

for i in range(5):
      print(f"Dimension {i+1}")

      kl_dim_tru_consist_0_i = kl_divergence(kde_truth_pca[i], kde_consist_0_pca[i], x_range)
      kl_dim_tru_consist_0.append(kl_dim_tru_consist_0_i)
      print(f"KL divergence (PC distribution) True vs Gamma_2 0 : {kl_dim_tru_consist_0_i}")

      kl_dim_tru_consist_01_i = kl_divergence(kde_truth_pca[i], kde_consist_01_pca[i], x_range)
      kl_dim_tru_consist_01.append(kl_dim_tru_consist_01_i)
      print(f"KL divergence (PC distribution) True vs Gamma_2 0.1 : {kl_dim_tru_consist_01_i}")

      kl_dim_tru_consist_03_i = kl_divergence(kde_truth_pca[i], kde_consist_03_pca[i], x_range)
      kl_dim_tru_consist_03.append(kl_dim_tru_consist_03_i)
      print(f"KL divergence (PC distribution) True vs Gamma_2 0.3 : {kl_dim_tru_consist_03_i}")

      kl_dim_tru_consist_05_i = kl_divergence(kde_truth_pca[i], kde_consist_05_pca[i], x_range)
      kl_dim_tru_consist_05.append(kl_dim_tru_consist_05_i)
      print(f"KL divergence (PC distribution) True vs Gamma_2 0.5 : {kl_dim_tru_consist_05_i}")

      kl_dim_tru_consist_1_i = kl_divergence(kde_truth_pca[i], kde_consist_1_pca[i], x_range)
      kl_dim_tru_consist_1.append(kl_dim_tru_consist_1_i)
      print(f"KL divergence (PC distribution) True vs Gamma_2 1 : {kl_dim_tru_consist_1_i}")
      



Dimension 1
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [5.36995503e-07 5.40557690e-07 5.44141462e-07 ... 4.38127133e-05
 4.38161257e-05 4.38195433e-05] q_x shape: 40000
KL divergence (PC distribution) True vs Gamma_2 0 : 158.8179909056119
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [0.00026947 0.00027021 0.00027094 ... 0.00033238 0.00033235 0.00033233] q_x shape: 40000
KL divergence (PC distribution) True vs Gamma_2 0.1 : 103.53885571905842
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [0.0012604  0.00126071 0.00126103 ... 0.00042726 0.00042684 0.00042642] q_x shape: 40000
KL divergence (PC distribution) True vs Gamma_2 0.3 : 30.64266137676418
p_x: [0.00076578 0.00076647 0.00076716 ... 0.0009838  0.0009832  0.0009826 ] p_x shape: 40000
q_x: [0.00066794 0.00066803 0.00066812 ... 0.00021954 0.00021935 0.00021916] q_x shape: 40